In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

In [2]:
#parameters
classes = 12
image_size = 1000
batch_size=4
train_total = 3820
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)

In [4]:
model_mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 1000, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 500, 500, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 500, 500, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 500, 500, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 500, 500, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 500, 500, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 500, 500, 16)      0         
__________

In [5]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 1000, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 500, 500, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 500, 500, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 500, 500, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 500, 500, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 500, 500, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 500, 500, 16)      0         
__________

In [8]:
print(model.layers[0].trainable)

True


In [9]:
model.load_weights('mobilenet_from_scratch_save_best_image_increase_1000.hdf5')

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [12]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [13]:
callbacks = [ModelCheckpoint(filepath='mobilenet_from_scratch_save_best_image_increase_400.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./08-metrics.csv')]

In [44]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[4,63,63,256]
	 [[Node: training_3/RMSprop/gradients/conv_dw_10_relu_2/clip_by_value/Minimum_grad/LessEqual = LessEqual[T=DT_FLOAT, _class=["loc:@conv_dw_10_relu_2/clip_by_value/Minimum"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv_dw_10_relu_2/Relu, conv1_relu_2/Cast)]]
	 [[Node: loss_4/mul/_2965 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6901_loss_4/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'training_3/RMSprop/gradients/conv_dw_10_relu_2/clip_by_value/Minimum_grad/LessEqual', defined at:
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/janardhan/anaconda3/envs/dl/lib/python3.5/runpy.py", line 85, in _run_c

In [23]:
model.load_weights('mobilenet_from_scratch_save_best_image_increase_400.hdf5')

In [14]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.023064078043745623, 0.99354838709677418]

In [17]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-5),
             metrics=['accuracy'])

In [18]:
callbacks = [ModelCheckpoint(filepath='mobilenet_from_scratch_save_best_image_increase_400_temp.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./08-metrics.csv')]

In [19]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.10915, saving model to mobilenet_from_scratch_save_best_image_increase_400_temp.hdf5
138s - loss: 0.0892 - acc: 0.9712 - val_loss: 0.1091 - val_acc: 0.9656
Epoch 2/100
Epoch 00001: val_loss improved from 0.10915 to 0.10178, saving model to mobilenet_from_scratch_save_best_image_increase_400_temp.hdf5
131s - loss: 0.0853 - acc: 0.9738 - val_loss: 0.1018 - val_acc: 0.9645
Epoch 3/100
Epoch 00002: val_loss improved from 0.10178 to 0.09651, saving model to mobilenet_from_scratch_save_best_image_increase_400_temp.hdf5
136s - loss: 0.0722 - acc: 0.9762 - val_loss: 0.0965 - val_acc: 0.9720
Epoch 4/100
Epoch 00003: val_loss improved from 0.09651 to 0.09299, saving model to mobilenet_from_scratch_save_best_image_increase_400_temp.hdf5
135s - loss: 0.0647 - acc: 0.9806 - val_loss: 0.0930 - val_acc: 0.9699
Epoch 5/100
Epoch 00004: val_loss did not improve
140s - loss: 0.0683 - acc: 0.9770 - val_loss: 0.1245 - val_acc: 0.9602
Epoch 6/100
Epo

KeyboardInterrupt: 

In [43]:
model.load_weights('mobilenet_from_scratch_save_best_image_increase_400_temp.hdf5')

In [15]:
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 930 images belonging to 12 classes.


In [16]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))

In [17]:
predictions_valid = np.argmax(predictions_valid, axis=1)


In [18]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [19]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

true_positive
[  45.   94.   64.  108.   45.   88.  131.   36.  107.   40.   95.   71.]
false_positive
[ 0.  1.  1.  0.  0.  0.  3.  0.  1.  0.  0.  0.]
false_negative
[ 2.  0.  1.  0.  0.  1.  0.  0.  0.  2.  0.  0.]
Total
[  47.   94.   65.  108.   45.   89.  131.   36.  107.   42.   95.   71.]


In [ ]:
predictions_valid

In [20]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [21]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [24]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_size_1000.csv', encoding="utf8", index=False)

In [25]:
from IPython.display import FileLink
FileLink('submission_mobilenet_size_1000.csv')

/home/janardhan/plant_seeding/submission_mobilenet_size_1000.csv